# Save data to website

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import pandas as pd

In [2]:
data_path = '../data/'
venue_short = 'neurips2022'
# date = time.strftime("%Y%m%d")
date = '20221105'
df = pd.read_csv(f'{data_path}{venue_short}_{date}.csv')

# get average rating for each paper from ratings column
df['ratings_avg'] = df['ratings'].apply(lambda x: np.mean(eval(x)))
df['ratings_std'] = df['ratings'].apply(lambda x: np.std(eval(x)))
df['confidence_avg'] = df['confidences'].apply(lambda x: np.mean(eval(x)))
df['confidence_std'] = df['confidences'].apply(lambda x: np.std(eval(x)))

# Write all submissions

In [3]:
df['url'] = df['id'].apply(lambda x: 'https://openreview.net/forum?id=' + x)
df.head()

,id,title,keywords,ratings,confidences,withdraw,review_lengths,decision,ratings_avg,ratings_std,confidence_avg,confidence_std,url
0,09QFnDWPF8,Statistical Learning and Inverse Problems: A S...,"['Statistical Learning', 'Inverse Problems', '...","[8, 5, 7]","[3, 3, 3]",0,"[875, 649, 214]",Accept,6.666667,1.247219,3.000000,0.000000,https://openreview.net/forum?id=09QFnDWPF8
1,pnSyqRXx73,Efficiency Ordering of Stochastic Gradient Des...,"['Stochastic Gradient Descent', 'Asymptotic An...","[5, 6, 7, 7]","[3, 3, 4, 4]",0,"[1280, 118, 287, 777]",Accept,6.250000,0.829156,3.500000,0.500000,https://openreview.net/forum?id=pnSyqRXx73
2,EqJ5_hZSqgy,Self-Aware Personalized Federated Learning,"['Federared Learning', 'Personalization']","[7, 7, 3]","[3, 3, 3]",0,"[209, 303, 221]",Accept,5.666667,1.885618,3.000000,0.000000,https://openreview.net/forum?id=EqJ5_hZSqgy
3,xnI37HyfoP,Nonnegative Tensor Completion via Integer Opti...,"['tensor completion', 'machine learning']","[7, 6, 6]","[4, 3, 4]",0,"[445, 269, 1083]",Accept,6.333333,0.471405,3.666667,0.471405,https://openreview.net/forum?id=xnI37HyfoP
4,OoNmOfYVhEU,TPU-KNN: K Nearest Neighbor Search at Peak FLOP/s,"['TPU', 'K-nearest neighbor search', 'Approxim...","[6, 5, 6, 6]","[4, 5, 4, 4]",0,"[162, 204, 446, 233]",Accept,5.750000,0.433013,4.250000,0.433013,https://openreview.net/forum?id=OoNmOfYVhEU


In [4]:
# Colored tags
rating_to_hex = {1: 'E62E30', 3: 'F18032', 5: 'FFD334', 6: '81BF2B', 8: '01AD23', 10: '01A7E5'}
rating_to_letters = {1: 'one', 2: 'one', 3: 'three', 4: 'three', 5: 'five', 6: 'five', 7: 'eight', 8: 'eight', 9: 'ten', 10: 'ten'}

# Write in a single row ratings with color mark given by rating
def write_ratings(ratings, opacity=0.6):
    if ratings == 'Ratings not available yet':
        return ratings
    ratings = ratings.strip('[]').split(',')
    # ratings_html = [f"{str(r).strip()}" for r in ratings]
    ratings_html = [f"{str(r).strip()}" for r in ratings]
    return ', '.join(ratings_html)

In [5]:
# read template 
with open('../web/openreview_template.html', 'r') as f:
    html_temp = f.readlines()

# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1


# write data
for i, el in df.iterrows():
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{el['url']}'> {el['title']}</a></td>" \
           f"<td class='r'>{el['ratings_avg']:.2f}</td" \
           f"<td data-sort='{el['ratings_avg']}'>" \
           f"<td data-sort='{len(el['ratings'])}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r'>{write_ratings(el['ratings'])}</td>" \
           f"<td class='r'>{el['decision']}</td" \
           f"</table></td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open(f'../submissions.html' ,'w') as f:
    f.write("".join(html_temp))